In [40]:

import numpy as np
import pandas as pd
import scipy.stats as stats

In [86]:
class NB():
    def prior(self, features, target):
        self.prior = (features.groupby(target).apply(lambda x: len(x)) / self.rows).to_numpy()
        return self.prior
    
    def parameters(self, features, target):
        self.mean = features.groupby(target).apply(np.mean).to_numpy()
        self.var = features.groupby(target).apply(np.var).to_numpy()
        return self.mean, self.var
    
    def gaussians(self, class_idx, x):     
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp((-1/2)*((x-mean)**2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        prob = numerator / denominator
        return prob
    
    def calc_posterior(self, x):
        posteriors = []
        for i in range(self.count):
            prior = np.log(self.prior[i])
            conditional = np.sum(np.log(self.gaussians(i, x)))
            posterior = prior + conditional
            posteriors.append(posterior)
        return self.classes[np.argmax(posteriors)]
     
    def fit(self, features, target):
        self.classes = np.unique(target)
        self.count = len(self.classes)
        self.feature_nums = features.shape[1]
        self.rows = features.shape[0]   
        self.parameters(features, target)
        self.prior(features, target)
        
    def predict(self, features):
        preds = [self.calc_posterior(f) for f in features.to_numpy()]
        return preds

    def accuracy(self, y_test, y_pred):
        accuracy = np.sum(y_test == y_pred) / len(y_test)
        return accuracy



In [43]:
df=pd.read_csv('/content/Sheet.csv')
X, y = df.iloc[:,1:], df.iloc[:, 0]
X_train, X_test, y_train, y_test = X[:6000], X[6000:], y[:6000], y[6000:]


In [87]:
x = NB()
x.fit(X_train, y_train)

In [88]:
predictions = x.predict(X_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in true_divide
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log


In [89]:
from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_test, predictions))


Classification Report

              precision    recall  f1-score   support

           0       0.10      1.00      0.18      6311
           1       0.00      0.00      0.00      7206
           2       0.00      0.00      0.00      6409
           3       0.00      0.00      0.00      6533
           4       0.00      0.00      0.00      6201
           5       0.00      0.00      0.00      5799
           6       0.00      0.00      0.00      6268
           7       0.00      0.00      0.00      6642
           8       0.00      0.00      0.00      6274
           9       0.00      0.00      0.00      6357

    accuracy                           0.10     64000
   macro avg       0.01      0.10      0.02     64000
weighted avg       0.01      0.10      0.02     64000



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
